In [ ]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json"
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json"
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab, 1.0.0-beta.21228.2"

Restore sources https://www.myget.org/F/gregs-experimental-packages/api/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json

Installed package Microsoft.DotNet.Interactive.ExtensionLab version 1.0.0-beta.21228.2

Loading extensions from `Microsoft.DotNet.Interactive.ExtensionLab.dll`

Create strongly-typed dataframes using #!linqify . 
 The #!linqify magic command replaces a Microsoft.Data.Analysis.DataFrame variable with a generated, strongly-typed data frame, allowing the use of LINQ operations over the contained data.

Inspect code compilation details using the #!inspect magic command. 
 The #!inspect magic command allows you to see the C# decompilation, IL, and JIT Asm for the code in a C# cell.

Explain things visually using the Mermaid language . 
 This extension adds a new kernel that can render Mermaid markdown. This code will render a sequence diagram: 
 
 
#!mermaid
sequenceDiagram
 participant Alice
 participant Bob
 Alice->>John: Hello John, how are you?
 loop Healthcheck
 John->>John: Fight against hypochondria
 end
 Note right of John: Rational thoughts prevail!
 John-->>Alice: Great!
 John->>Bob: How about you?
 Bob-->>John: Jolly good!
 
 
 This extension also adds gestures to render a class diagram from any type. Use the ExploreWithUmlClassDiagram().Display(); extension method on System.Type to render its class diagram. 

 
 
typeof(List ).ExploreWithUmlClassDiagram().Display();

Explore data visually using the nteract Data Explorer . 
 This extension adds the ability to sort, filter, and visualize data using the nteract Data Explorer . Use the ExploreWithNteract() extension method with variables of type JsonElement , IEnumerable or IDataView to render the data explorer. 
 
 
 using Microsoft.Data.Analysis;
 using System.Collections.Generic;
 using Microsoft.ML;

 var dataFrame = DataFrame.LoadCsv("./Data.csv");

 dataFrame.ExploreWithNteract().Display();

Use the #!record magic command to keep a transcript of the code you run. 
 Once you enable transcripts using #!record , each code submission (including re-running cells) is recorded in the specified file. The JSON format used is the same format recognized by the .NET Interactive stdio and http APIs and can be used to replay a notebook session via automation.

Explore data visually using the SandDance Explorer . 
 This extension adds the ability to sort, filter, and visualize data using the SandDance Explorer . Use the ExploreWithSandDance() extension method with variables of type JsonElement , IEnumerable or IDataView to render the data explorer. 
 
 
 using Microsoft.Data.Analysis;
 using System.Collections.Generic;
 using Microsoft.ML;

 var dataFrame = DataFrame.LoadCsv("./Data.csv");

 dataFrame.ExploreWithSandDance().Display();

Query SQLite databases. 
 This extension adds support for connecting to SQLite databases using the #!connect sqlite magic command. For more information, run a cell using the #!sql magic command.

Run this cell if you don't already have the SQLite data base containing the energy data set.

In [ ]:
open System
open System.Net.Http
open System.IO

let client = new HttpClient()

let dataPackage =
    client.GetAsync("https://data.open-power-system-data.org/time_series/2020-10-06/time_series.sqlite")
    |> Async.AwaitTask
    |> Async.RunSynchronously

let copyToTarget (targetPath: string) (msg: HttpResponseMessage) =
    use fileStream = new FileStream(targetPath, FileMode.CreateNew)
    use contentStream = msg.Content.ReadAsStream()
    contentStream.CopyTo(fileStream)

let sqliteDbPath = "../data/time_series.sqlite"

copyToTarget sqliteDbPath dataPackage

In [ ]:
#!connect sqlite --kernel-name energytimeseries "Data Source=../data/time_series.sqlite"

Kernel added: #!sql-energytimeseries

In [ ]:
#!sql-energytimeseries
select name from sqlite_master
where type in ('table', 'view')
and name not like 'sqlite_%'
order by 1;

In [ ]:
#!sql-energytimeseries
select name from PRAGMA_TABLE_INFO("time_series_60min_singleindex");

In [ ]:
#!sql-energytimeseries
select utc_timestamp, AT_load_actual_entsoe_transparency
from time_series_60min_singleindex
where AT_load_actual_entsoe_transparency is not null
limit 1000;

In [ ]:
#!sql-energytimeseries
select utc_timestamp, AT_load_actual_entsoe_transparency
from time_series_15min_singleindex
where AT_load_actual_entsoe_transparency is not null
limit 4000;

In [ ]:
#!sql-energytimeseries
select count(1)
from time_series_60min_singleindex
where AT_load_actual_entsoe_transparency is not null;

In [ ]:
#!sql-energytimeseries
select utc_timestamp, cast(strftime('%Y', utc_timestamp) as int) as yr, AT_load_actual_entsoe_transparency
from time_series_60min_singleindex
where AT_load_actual_entsoe_transparency is not null
and yr < 2020
and yr > 2017;

In [ ]:
#r "nuget: Microsoft.Data.Sqlite, 5.0.5"
#r "nuget: Ply, 0.3.1"

Installed package Microsoft.Data.Sqlite version 5.0.5

Installed package Ply version 0.3.1

In [ ]:
open Microsoft.Data.Sqlite
open FSharp.Control.Tasks.NonAffine

let connectionString = "Data Source=../data/time_series.sqlite"

type RowRecord =
    { TimeStamp: DateTime
      Value: int }

let getSeries () = task {
    use connection = new SqliteConnection(connectionString)
    do! connection.OpenAsync()

    let command = connection.CreateCommand()
    command.CommandText <-
        """
        select utc_timestamp, cast(strftime('%Y', utc_timestamp) as int) as yr, AT_load_actual_entsoe_transparency
        from time_series_60min_singleindex
        where AT_load_actual_entsoe_transparency is not null
        """

    use! reader = command.ExecuteReaderAsync()
    let rows = ResizeArray<RowRecord>()
    while reader.Read() do
        let newRecord = { TimeStamp = reader.GetDateTime(0); Value = reader.GetInt32(2) }
        rows.Add(newRecord)

    return (Seq.toList rows)
}

let austriaLoad =
    getSeries()
    |> Async.AwaitTask
    |> Async.RunSynchronously

display (List.length austriaLoad)

austriaLoad
|> List.take 10

50400

index,TimeStamp,Value
0,2015-01-01 01:00:00Z,5946
1,2015-01-01 02:00:00Z,5726
2,2015-01-01 03:00:00Z,5347
3,2015-01-01 04:00:00Z,5249
4,2015-01-01 05:00:00Z,5309
5,2015-01-01 06:00:00Z,5574
6,2015-01-01 07:00:00Z,5925
7,2015-01-01 08:00:00Z,6343
8,2015-01-01 09:00:00Z,6882
9,2015-01-01 10:00:00Z,6963


In [ ]:
austriaLoad
|> List.distinct
|> List.length

50400

In [ ]:
let austriaLoadUnique =
    austriaLoad
    |> List.distinctBy (fun row -> row.TimeStamp)

austriaLoadUnique
|> List.length

50395

In [ ]:
#r "nuget: Deedle, 2.3.0"
#r "nuget: Plotly.NET, 2.0.0-beta8"
#r "nuget: Plotly.NET.Interactive, 2.0.0-beta8"

#i "nuget:https://www.myget.org/F/gregs-experimental-packages/api/v3/index.json"
#r "nuget:Deedle.DotNet.Interactive.Extension, 0.1.0-alpha5"

Restore sources https://www.myget.org/F/gregs-experimental-packages/api/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json

In [ ]:
open Deedle

let data =
    austriaLoadUnique
    |> Frame.ofRecords
    |> Frame.indexRowsDate "TimeStamp"

In [ ]:
let minDate =
    data.RowKeys
    |> Seq.min

let maxDate =
    data.RowKeys
    |> Seq.max

let hoursBetween = (maxDate - minDate).TotalHours
let numOfValues = hoursBetween + 1.

let timeSeriesIndex = [ for x in 0. .. hoursBetween -> minDate.AddHours(x)]
let tsTicks = [ 1. .. numOfValues ] // I only really need the running row count for debugging

let frameSeries =
    Seq.zip timeSeriesIndex tsTicks
    |> Series.ofObservations

let baseFrame = Frame.ofColumns [ "Ticks" => frameSeries ]

let missingCount (frame: Frame<'K, 'C>) =
    frame.Columns
    |> Series.map (fun _ os -> os
                            |> Series.filterAll (fun _ v -> v.IsNone)
                            |> Series.fillMissingWith 0.
                            |> Stats.count
                            |> fun count -> Series.ofObservations [ "Missing Count", float count
                                                                    "Missing %", (float count) / (float os.KeyCount) * 100. ])
    |> Frame.ofColumns
    |> Frame.transpose

let completeFrame = baseFrame.Join(data, JoinKind.Left, Lookup.Exact)
missingCount completeFrame

,Missing Count,Missing %
,System.Double,System.Double
Ticks,0,0
Value,6,"0,011904525703855083"


Funny observation! Why is that? We read the values in UTC. Apparently we dropped the "double" values when switching to summer time.

In [ ]:
completeFrame
|> Frame.filterRows (fun _ os -> os.TryGetAs<float>("Value").HasValue |> not)

,Ticks,Value
,System.Double,System.Object
29.03.2015 02:00:00,2090,N/A
27.03.2016 02:00:00,10826,N/A
26.03.2017 02:00:00,19562,N/A
25.03.2018 02:00:00,28298,N/A
31.03.2019 02:00:00,37202,N/A
29.03.2020 02:00:00,45938,N/A


In [ ]:
completeFrame?Value <-
    completeFrame?Value
    |> Series.fillMissing Direction.Forward

missingCount completeFrame

,Missing Count,Missing %
,System.Double,System.Double
Ticks,0,0
Value,0,0


In [ ]:
let describeFrame (frame: Frame<'R, 'C>) =
    Frame.getNumericCols frame
    |> Series.map (fun key s -> Stats.describe s)
    |> Frame.ofRows
    |> Frame.transpose

describeFrame completeFrame

,Ticks,Value
,System.Double,System.Double
unique,50401,6268
mean,25201,"7070,174183051923"
std,"14549,659795564521","1400,1912775783496"
min,1,664
0.25,12601,5941
0.5,25201,7032
0.75,37801,8079
max,50401,10803


In [ ]:
completeFrame.SaveCsv("../data/at_load_hourly_mw.csv", [ "TimeStamp" ])